# rfe, pca, regression

In [1]:
import pymysql.cursors
import pandas as pd

In [2]:
conn = pymysql.connect(host='localhost', user='me', 
                       password='1234', db='tip', charset='utf8',
                       autocommit=True, cursorclass=pymysql.cursors.DictCursor)
try:
    with conn.cursor() as curs:
        sql = "select * from tips;"
        curs.execute(sql)
        rs = curs.fetchall()
        df = pd.DataFrame(rs)
finally:
    conn.close()

In [3]:
df["sex"] = df["sex"].replace({"Female" : 0, "Male" : 1})
df["smoker"] = df["smoker"].replace({"No" : 0, "Yes" : 1})
df["day"] = df["day"].replace({"Thur" : 0, "Fri" : 1, "Sat" : 2, "Sun" : 3})
df["time"] = df["time"].replace({"Lunch" : 0, "Dinner" : 1})
df['tip_rate'] = df['tip']/df['total_bill']

In [4]:
df.describe()

,total_bill,tip,sex,smoker,day,time,size,tip_rate
count,249.000000,249.000000,247.000000,249.000000,249.000000,248.000000,248.000000,249.000000
mean,19.622972,2.968795,0.643725,0.373494,1.751004,0.725806,2.572581,0.160391
std,8.893766,1.385573,0.479870,0.484706,1.158006,0.447009,0.945249,0.060794
min,3.070000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.035638
25%,13.160000,2.000000,0.000000,0.000000,1.000000,0.000000,2.000000,0.129422
50%,17.510000,2.750000,1.000000,0.000000,2.000000,1.000000,2.000000,0.155581
75%,24.060000,3.510000,1.000000,1.000000,3.000000,1.000000,3.000000,0.190476
max,50.810000,10.000000,1.000000,1.000000,3.000000,1.000000,6.000000,0.710345


In [5]:
df.isnull().sum()

total_bill    0
tip           0
sex           2
smoker        0
day           0
time          1
size          1
tip_rate      0
dtype: int64

# null값 처리

In [6]:
from numpy import isnan
from sklearn.impute import SimpleImputer

In [7]:
y = df['tip']
x = df.drop('tip',axis =1)

In [8]:
data = x.values
y = y.values

In [9]:
sum(isnan(data).flatten())

4

In [10]:
sum(isnan(y).flatten())

0

In [11]:
imputer = SimpleImputer(strategy = 'median')
imputer.fit(data)
data_trans = imputer.transform(data)

In [12]:
df1 = pd.DataFrame(data_trans)

In [13]:
df1.isnull().sum()

0    0
1    0
2    0
3    0
4    0
5    0
6    0
dtype: int64

In [14]:
df1.columns = ['total_bill', "sex", 'smoker',	'day', 'time', 'size', 'tip_rate']

In [15]:
df1

,total_bill,sex,smoker,day,time,size,tip_rate
0,16.99,0.0,0.0,3.0,1.0,2.0,0.059447
1,10.34,1.0,0.0,3.0,1.0,3.0,0.160542
2,21.01,1.0,0.0,3.0,1.0,3.0,0.166587
3,23.68,1.0,0.0,3.0,1.0,2.0,0.139780
4,24.59,0.0,0.0,3.0,1.0,4.0,0.146808
...,...,...,...,...,...,...,...
244,16.99,0.0,0.0,3.0,1.0,2.0,0.059447
245,10.34,1.0,0.0,3.0,1.0,3.0,0.160542
246,10.34,1.0,0.0,3.0,1.0,3.0,0.160542
247,10.34,1.0,0.0,3.0,1.0,3.0,0.160542


## rfe

In [16]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVR

In [17]:
rfe = RFE(estimator=SVR(kernel="linear"), n_features_to_select=4)

In [18]:
# fit RFE
rfe.fit(df1, y)

RFE(estimator=SVR(kernel='linear'), n_features_to_select=4)

In [19]:
df1.shape

(249, 7)

In [20]:
for i in range(df1.shape[1]):
  print('Column: %d, Selected=%s, Rank: %d' % (i, rfe.support_[i], rfe.ranking_[i]))

Column: 0, Selected=True, Rank: 1
Column: 1, Selected=True, Rank: 1
Column: 2, Selected=True, Rank: 1
Column: 3, Selected=False, Rank: 4
Column: 4, Selected=False, Rank: 3
Column: 5, Selected=False, Rank: 2
Column: 6, Selected=True, Rank: 1


In [21]:
df1

,total_bill,sex,smoker,day,time,size,tip_rate
0,16.99,0.0,0.0,3.0,1.0,2.0,0.059447
1,10.34,1.0,0.0,3.0,1.0,3.0,0.160542
2,21.01,1.0,0.0,3.0,1.0,3.0,0.166587
3,23.68,1.0,0.0,3.0,1.0,2.0,0.139780
4,24.59,0.0,0.0,3.0,1.0,4.0,0.146808
...,...,...,...,...,...,...,...
244,16.99,0.0,0.0,3.0,1.0,2.0,0.059447
245,10.34,1.0,0.0,3.0,1.0,3.0,0.160542
246,10.34,1.0,0.0,3.0,1.0,3.0,0.160542
247,10.34,1.0,0.0,3.0,1.0,3.0,0.160542


In [22]:
df1.corr()

,total_bill,sex,smoker,day,time,size,tip_rate
total_bill,1.000000,0.131455,0.098592,0.150184,0.169485,0.588716,-0.331579
sex,0.131455,1.000000,-0.002302,0.233269,0.206811,0.090820,-0.059502
smoker,0.098592,-0.002302,1.000000,-0.049162,0.044676,-0.132500,0.035693
day,0.150184,0.233269,-0.049162,1.000000,0.874082,0.163612,0.005931
time,0.169485,0.206811,0.044676,0.874082,1.000000,0.103127,-0.037749
size,0.588716,0.090820,-0.132500,0.163612,0.103127,1.000000,-0.137625
tip_rate,-0.331579,-0.059502,0.035693,0.005931,-0.037749,-0.137625,1.000000


# pca

In [23]:
from sklearn.decomposition import PCA

In [24]:
df1['tip_rate'] = df1['tip_rate'] * 100

#### 표준화

In [25]:
from sklearn.preprocessing import StandardScaler

In [26]:
df_norm = StandardScaler().fit_transform(df1)

In [27]:
df_norm = pd.DataFrame(df_norm)

### ==== pca 이해 ===
 
##### 참고 ] https://chancoding.tistory.com/53
##### https://steadiness-193.tistory.com/245

In [77]:
trans = PCA()

X_dim = trans.fit_transform(df_norm)

df2 = pd.DataFrame(X_dim)

In [90]:
trans.components_[0]

array([ 0.40290512,  0.28064129, -0.01522685,  0.54861572,  0.54134026,
        0.36282748, -0.18166951])

In [80]:
trans.explained_variance_
# eigen_vector 값

array([2.20858836, 1.53956037, 1.05305951, 0.90851231, 0.86573221,
       0.33572903, 0.117044  ])

In [82]:
trans.explained_variance_ratio_

array([0.3142455 , 0.21905391, 0.14983291, 0.12926624, 0.12317934,
       0.04776867, 0.01665342])

### ==== pca 이해 ===

In [86]:
trans = PCA(n_components=4)

X_dim = trans.fit_transform(df_norm)

df2 = pd.DataFrame(X_dim)

In [87]:
df2['tip'] = y

In [88]:
df2.head(1)

,0,1,2,3,tip
0,0.520002,0.291243,-0.629332,0.360651,1.01


In [89]:
trans.explained_variance_ratio_

array([0.3142455 , 0.21905391, 0.14983291, 0.12926624])

# regression

In [34]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

In [35]:
fs = SelectKBest(score_func=f_regression, k=4)
# apply feature selection
X_selected = fs.fit_transform(df1, y)
print(X_selected.shape)

(249, 4)


In [36]:
fs.get_support(indices=True) # 선택된 피쳐

array([0, 3, 5, 6], dtype=int64)

In [37]:
df3 = pd.DataFrame(X_selected)

In [38]:
df1.head(1)

,total_bill,sex,smoker,day,time,size,tip_rate
0,16.99,0.0,0.0,3.0,1.0,2.0,5.944673


In [39]:
df3

,0,1,2,3
0,16.99,3.0,2.0,5.944673
1,10.34,3.0,3.0,16.054159
2,21.01,3.0,3.0,16.658734
3,23.68,3.0,2.0,13.978041
4,24.59,3.0,4.0,14.680765
...,...,...,...,...
244,16.99,3.0,2.0,5.944673
245,10.34,3.0,3.0,16.054159
246,10.34,3.0,3.0,16.054159
247,10.34,3.0,3.0,16.054159


In [40]:
df3.corr()

,0,1,2,3
0,1.000000,0.150184,0.588716,-0.331579
1,0.150184,1.000000,0.163612,0.005931
2,0.588716,0.163612,1.000000,-0.137625
3,-0.331579,0.005931,-0.137625,1.000000
